In [1]:
desired_cube_side = 512
chunk_size = 64
# raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF'
# Has issues on SciServer Compute
use_dask = False
dest_folder_name = "sabl2048b"
# timestep_nr = 1
# timestep_range = range(2,6)

In [2]:
%cd /home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing

/home/idies/workspace/Storage/ariel4/persistent/ncar-zarr-code/zarr_writing


In [3]:
import xarray as xr
import os
from utils import write_tools
import numpy as np
import morton, math

In [4]:
z_order = np.array(write_tools.morton_order_cube(4))

In [5]:
z_order[0,:,:]

array([[ 0,  4, 32, 36],
       [ 2,  6, 34, 38],
       [16, 20, 48, 52],
       [18, 22, 50, 54]])

In [6]:
data_xr = xr.open_dataset("/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt/jhd.001.nc")

In [7]:
dims = [dim for dim in data_xr.dims]
dims.reverse()
dims

['nnx', 'nny', 'nnz']

In [8]:
# Split 2048^3 into smaller 512^3 arrays
dims = [dim for dim in data_xr.dims]
smaller_groups = write_tools.split_zarr_group(data_xr, desired_cube_side, dims)

In [9]:
ds = data_xr
smaller_size = desired_cube_side
i=0
j=0
k=0


chunk = ds.isel( # TODO make this into a function: slice_group()
    {dims[0]: slice(0, 512),
     dims[1]: slice(0, 512),
     dims[2]: slice(512, 1024)}
)

In [11]:
np.testing.assert_array_equal(
    chunk['e'],#.to_array,
    smaller_groups[0][0][1]['e']
)

In [12]:
len(smaller_groups[0][0])

4

In [13]:
smaller_groups[0][0][0]['e'].shape

(512, 512, 512)

In [14]:
a = tuple(element * desired_cube_side for element in (1,1,1))
a

(512, 512, 512)

In [16]:
array_cube_side = 2048


bits = int(math.log(array_cube_side, 2))
mortoncurve = morton.Morton(dimensions = 3, bits = bits)

mortoncurve.pack(0,0,1)
mortoncurve.unpack(134217727)

[511, 511, 511]

In [17]:
mortoncurve.unpack(134217728)

[512, 0, 0]

In [18]:
mortoncurve.pack(1, 0, 0) * 512 ** 3

134217728

In [21]:
mike_dict = {}
for i in range(len(smaller_groups)):
    for j in range(len(smaller_groups[i])):
        for k in range(len(smaller_groups[i][j])):
            min_coord = tuple(element * desired_cube_side for element in (i,j,k))
            max_coord = tuple((element * desired_cube_side)-1 for element in (i+1,j+1,k+1))
            
            point_on_cube = (i * 4 * 4) + (j * 4) + k # 0 to 63
#             print(point_on_cube + 1, (i,j,k))
            
            mike_dict[dest_folder_name + str(point_on_cube + 1).zfill(2)] = (write_tools.morton_pack(2048, min_coord[0], min_coord[1], min_coord[2]), write_tools.morton_pack(2048, max_coord[0], max_coord[1], max_coord[2]))
#             print(write_tools.morton_pack(2048, min_coord[0], min_coord[1], min_coord[2]))
# print(mike_dict)

In [22]:
print(mike_dict)

{'sabl2048b01': (0, 134217727), 'sabl2048b02': (536870912, 671088639), 'sabl2048b03': (4294967296, 4429185023), 'sabl2048b04': (4831838208, 4966055935), 'sabl2048b05': (268435456, 402653183), 'sabl2048b06': (805306368, 939524095), 'sabl2048b07': (4563402752, 4697620479), 'sabl2048b08': (5100273664, 5234491391), 'sabl2048b09': (2147483648, 2281701375), 'sabl2048b10': (2684354560, 2818572287), 'sabl2048b11': (6442450944, 6576668671), 'sabl2048b12': (6979321856, 7113539583), 'sabl2048b13': (2415919104, 2550136831), 'sabl2048b14': (2952790016, 3087007743), 'sabl2048b15': (6710886400, 6845104127), 'sabl2048b16': (7247757312, 7381975039), 'sabl2048b17': (134217728, 268435455), 'sabl2048b18': (671088640, 805306367), 'sabl2048b19': (4429185024, 4563402751), 'sabl2048b20': (4966055936, 5100273663), 'sabl2048b21': (402653184, 536870911), 'sabl2048b22': (939524096, 1073741823), 'sabl2048b23': (4697620480, 4831838207), 'sabl2048b24': (5234491392, 5368709119), 'sabl2048b25': (2281701376, 2415919103